## ex.) of another similar idea for tracking tasks

In [ ]:
import os

# File to store tasks
TASKS_FILE = "tasks.txt"

# Load tasks from file
def load_tasks():
    if os.path.exists(TASKS_FILE):
        with open(TASKS_FILE, "r") as file:
            tasks = [line.strip() for line in file.readlines()]
    else:
        tasks = []
    return tasks

# Save tasks to file
def save_tasks(tasks):
    with open(TASKS_FILE, "w") as file:
        file.writelines(task + "\n" for task in tasks)

# Add a task
def add_task(task, tasks):
    tasks.append(task)
    save_tasks(tasks)
    print(f"Task '{task}' added!")

# Complete a task
def complete_task(index, tasks):
    if 0 <= index < len(tasks):
        print(f"Task '{tasks[index]}' completed!")
        tasks.pop(index)
        save_tasks(tasks)
    else:
        print("Invalid task number!")

# View tasks
def view_tasks(tasks):
    if not tasks:
        print("No tasks at the moment.")
    else:
        print("Your tasks:")
        for i, task in enumerate(tasks):
            print(f"{i + 1}. {task}")

# Main function
def main():
    tasks = load_tasks()

    while True:
        print("\n1. View Tasks\n2. Add Task\n3. Complete Task\n4. Exit")
        choice = input("Choose an option: ")

        if choice == '1':
            view_tasks(tasks)
        elif choice == '2':
            task = input("Enter a task: ")
            add_task(task, tasks)
        elif choice == '3':
            view_tasks(tasks)
            index = int(input("Enter task number to complete: ")) - 1
            complete_task(index, tasks)
        elif choice == '4':
            break
        else:
            print("Invalid option, please try again.")

if __name__ == "__main__":
    main()

## Commiting new data from csv to database 

In [12]:
import mysql.connector
import pandas as pd
import numpy as np

# For updating database, print new file:
FILE = '6-6-2024  9-6-2-24.CSV' # <----

def get_database_connection(): # to mysql
    connection = mysql.connector.connect(host='localhost', user='root', passwd='braedon123', database='budgettracker')
    return connection

connection = get_database_connection()
cursor = connection.cursor()

# one line of code that turns this entire file an easy peasy dataframe to manipulate
df = pd.read_csv(FILE)

# reformatting/cleaning data
df["Credit"] = pd.to_numeric(df['Credit'], errors='coerce')
df['Transaction Date'] = pd.to_datetime(df['Transaction Date'], format='%m/%d/%Y', errors='coerce')
df = df[df['Transaction Date'].notna()]

# with the db structured how we need it to be, we can make a new db with dates of every credit transaction
df_creditTrans = df[df['Credit'] > 0]

# putting data in expenses table
for index, row in df_creditTrans.iterrows():
    sql = """INSERT INTO deposits (source, amount, date) VALUES (%s, %s, %s)"""
    values = (
        str(row['Description']),
        float(row['Credit']),
        str(row['Transaction Date'])
    )
    cursor.execute(sql, values)
    
# checking the database to check that the data got inserting correctly
cursor.execute("SELECT * FROM deposits")
results = cursor.fetchall()
print('Deposits Data Table: id, source, amount, date \n ---------------------------')
for row in results:
    print(row[0], row[1], row[2], row[3])

cursor.close()
connection.close()

Deposits Data Table: id, source, amount, date 
 ---------------------------
65 Federal Express 255.32 2024-08-07
66 Federal Express 261.2 2024-08-14
67 Aldi 761.28 2024-08-16
68 FedEx 292.05 2024-08-22
69 Aldi 363.95 2024-08-30
70 EB From Savings 4686 300.0 2024-09-03
71 Federal Express 296.11 2024-09-04
72 EB From Savings 4686 985.0 2024-09-04


In [17]:
import mysql.connector

def get_database_connection(): # to mysql
    connection = mysql.connector.connect(host='localhost', user='root', passwd='braedon123', database='budgettracker')
    return connection

try:
    connection = get_database_connection()
    cursor = connection.cursor()
    cursor.execute("SELECT * FROM deposits")
    results = cursor.fetchall()
    for row in results:
        print(row[:])
except mysql.connector.Error as err:
    print(f"Error: {err}")
finally:
    if connection.is_connected():
        cursor.close()
        connection.close()